In [1]:
device = 'cuda:1'

num_classes = 2

from models import get_instance_segmentation_model
model = get_instance_segmentation_model(num_classes)

import torch
model_path = './models/mrcnn_cp_400.pth'
# model_path = './models/mrcnn_800.pth'
model_dict = torch.load(model_path)
model.load_state_dict(model_dict['state_dict'])

<All keys matched successfully>

In [2]:
import sys

cp_dir = '/home/epsilon/workspace/copy-paste-aug'
sys.path.insert(0,cp_dir)

import torch
import utils

batch_size = 2
device = 'cuda' if torch.cuda.is_available else 'cpu'

import albumentations as A
from albumentations.pytorch import ToTensorV2


valtf = A.Compose([
                  A.Normalize(),
                  ToTensorV2()
                  ], bbox_params=A.BboxParams(format='coco')
                    )

valtf0 = A.Compose([
                A.Normalize(),
                ToTensorV2(),
                ], bbox_params=A.BboxParams(format='coco', label_fields=['class_labels'])
                )

bcad_img = './images'
bcad_ann = './annotations/v1v2_ann.json'


from dataset import BCAD_CP_dt

dataset = BCAD_CP_dt(root=bcad_img,annFile=bcad_ann, mode='train', transforms=valtf)

from dataset import BurnedCAD_is

dataset0 = BurnedCAD_is(root=bcad_img,annFile=bcad_ann, mode='train', transform=valtf0)


import torch

trdl = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)
trdl0 = torch.utils.data.DataLoader(
    dataset0, batch_size=batch_size, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
from engine import evaluate
model.to(device)
evaluator = evaluate(model, trdl0, device=device)

creating index...
index created!
Test:  [ 0/29]  eta: 0:00:31  model_time: 0.9569 (0.9569)  evaluator_time: 0.0291 (0.0291)  time: 1.0934  data: 0.1014  max mem: 1038
Test:  [28/29]  eta: 0:00:00  model_time: 0.1524 (0.1875)  evaluator_time: 0.0185 (0.0238)  time: 0.2234  data: 0.0064  max mem: 1614
Test: Total time: 0:00:06 (0.2248 s / it)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.326
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.700
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.214
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.335
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.342
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all |

In [9]:
import os
model_name = os.path.splitext(os.path.basename(model_path))[0]
metric_types = ['ap','ap50','ap75','Maskap','Maskap50','Maskap75']

In [10]:
import pandas as pd
num_epoch = -1
aps = []

bbox = evaluator.coco_eval['bbox'].stats[:3]
segm = evaluator.coco_eval['segm'].stats[:3]
row = bbox.tolist()+segm.tolist()
aps.append(row)

import dataframe_image as dfi
df = pd.DataFrame(aps,index=[model_name],columns=metric_types)
dfi.export(df,f'./assets/{model_name}_table.png')

[0228/172130.796344:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0228/172130.924420:ERROR:angle_platform_impl.cc(43)] DisplayVkXcb.cpp:59 (initialize): xcb_connect() failed, error 1
ERR: DisplayVkXcb.cpp:59 (initialize): xcb_connect() failed, error 1
[0228/172130.924510:ERROR:angle_platform_impl.cc(43)] Display.cpp:1021 (initialize): ANGLE Display::initialize error 12289: Not initialized.
ERR: Display.cpp:1021 (initialize): ANGLE Display::initialize error 12289: Not initialized.
[0228/172130.924545:ERROR:gl_display.cc(508)] EGL Driver message (Critical) eglInitialize: Not initialized.
[0228/172130.924565:ERROR:gl_display.cc(932)] eglInitialize SwANGLE failed with error EGL_NOT_INITIALIZED
[0228/172130.924593:ERROR:gl_ozone_egl.cc(23)] GLDisplayEGL::Initialize failed.
[0228/172130.925302:ERROR:viz_main_impl.cc(186)] Exiting GPU process due to errors during initialization
[0228/172130.930207:ERROR:gpu_init.cc(523)] Passthrough is not su

In [11]:
df

,ap,ap50,ap75,Maskap,Maskap50,Maskap75
mrcnn_cp_400,0.326143,0.70013,0.214265,0.235365,0.701493,0.100998


In [110]:
from engine import evaluate
model.to(device)
evaluator = evaluate(model, trdl, device=device)

IndexError: list index out of range